In [22]:
from copy import deepcopy
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.layers import * 
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost, MSECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed
import numpy
import logging
from mlp.dataset import *
from base import *
from itertools import chain, combinations
from collections import OrderedDict

In [23]:
def powerset(iterable):
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

In [24]:
devices = ['RLAx', 'LLAx', 'RHAx', 'LHAx']
p =map(lambda x: list(x) if len(x) > 1 else list(x)[0], filter(lambda x: len(x) >0 ,powerset(devices)))
#p.pop(0)
print p, len(p)

['RLAx', 'LLAx', 'RHAx', 'LHAx', ['RLAx', 'LLAx'], ['RLAx', 'RHAx'], ['RLAx', 'LHAx'], ['LLAx', 'RHAx'], ['LLAx', 'LHAx'], ['RHAx', 'LHAx'], ['RLAx', 'LLAx', 'RHAx'], ['RLAx', 'LLAx', 'LHAx'], ['RLAx', 'RHAx', 'LHAx'], ['LLAx', 'RHAx', 'LHAx'], ['RLAx', 'LLAx', 'RHAx', 'LHAx']] 15


In [25]:
mod = OrderedDict()
for xn in p:
    
    rng = numpy.random.RandomState([2015,10,10])
    train_dp = MACLDataProvider(dset='train', batch_size=100,
                                max_num_batches=-10, randomize=True,name=deepcopy(xn),
                                conv_reshape=False, fft=True)
    valid_dp = MACLDataProvider(dset='valid', batch_size=1140,
                                max_num_batches=1, randomize=False,name=deepcopy(xn),
                                conv_reshape=False, fft=True)
    
    learning_rate =0.01
    max_epochs = 1000
    if isinstance(xn, str):
        dim = 125
    else:
        dim = len(xn)*125
    cost = CECost()    
    stats = list()

    train_dp.reset()
    valid_dp.reset()
    
    model = MLP(cost=cost)

    model.add_layer(Sigmoid(idim=dim, odim=dim, irange=1.6, rng=rng))

    model.add_layer(Softmax(idim=dim, odim=19, rng=rng))
    print xn, "x"
    lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
    optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

    logger.info('Training started...')
    tr_stats_f, valid_stats_f = optimiser.train(model, train_dp, valid_dp)
    mod[str(xn)] = (deepcopy(model), deepcopy(valid_dp))
    print xn, "x"
    


(6840, 45, 125) (6840,)
(6840, 125)
(1140, 45, 125) (1140,)
(1140, 125)
RLAx x
RLAx x
(6840, 45, 125) (6840,)
(6840, 125)
(1140, 45, 125) (1140,)
(1140, 125)
LLAx x
LLAx x
(6840, 45, 125) (6840,)
(6840, 125)
(1140, 45, 125) (1140,)
(1140, 125)
RHAx x
RHAx x
(6840, 45, 125) (6840,)
(6840, 125)
(1140, 45, 125) (1140,)
(1140, 125)
LHAx x
LHAx x
(6840, 45, 125) (6840,)
shape final:  (6840, 250)
(6840, 250)
(1140, 45, 125) (1140,)
shape final:  (1140, 250)
(1140, 250)
['RLAx', 'LLAx'] x
['RLAx', 'LLAx'] x
(6840, 45, 125) (6840,)
shape final:  (6840, 250)
(6840, 250)
(1140, 45, 125) (1140,)
shape final:  (1140, 250)
(1140, 250)
['RLAx', 'RHAx'] x
['RLAx', 'RHAx'] x
(6840, 45, 125) (6840,)
shape final:  (6840, 250)
(6840, 250)
(1140, 45, 125) (1140,)
shape final:  (1140, 250)
(1140, 250)
['RLAx', 'LHAx'] x
['RLAx', 'LHAx'] x
(6840, 45, 125) (6840,)
shape final:  (6840, 250)
(6840, 250)
(1140, 45, 125) (1140,)
shape final:  (1140, 250)
(1140, 250)
['LLAx', 'RHAx'] x
['LLAx', 'RHAx'] x
(6840, 4

In [26]:
import cPickle as p

In [27]:
with open("shapley2.pkl", "wb") as f:
    p.dump(mod, f)

In [28]:
mod

OrderedDict([('RLAx',
              (<mlp.layers.MLP at 0x31c5f50>,
               <mlp.dataset.MACLDataProvider at 0x677c910>)),
             ('LLAx',
              (<mlp.layers.MLP at 0x677c4d0>,
               <mlp.dataset.MACLDataProvider at 0x677ca50>)),
             ('RHAx',
              (<mlp.layers.MLP at 0x677c6d0>,
               <mlp.dataset.MACLDataProvider at 0x677ccd0>)),
             ('LHAx',
              (<mlp.layers.MLP at 0x677c610>,
               <mlp.dataset.MACLDataProvider at 0x677ced0>)),
             ("['RLAx', 'LLAx']",
              (<mlp.layers.MLP at 0x677c590>,
               <mlp.dataset.MACLDataProvider at 0x6779090>)),
             ("['RLAx', 'RHAx']",
              (<mlp.layers.MLP at 0x677ce50>,
               <mlp.dataset.MACLDataProvider at 0x6779290>)),
             ("['RLAx', 'LHAx']",
              (<mlp.layers.MLP at 0x677cf50>,
               <mlp.dataset.MACLDataProvider at 0x6779490>)),
             ("['LLAx', 'RHAx']",
              (<mlp.

In [29]:
f2 = open("shapley2.pkl")
g = p.load(f2)

In [30]:
g

OrderedDict([('RLAx',
              (<mlp.layers.MLP at 0x579ea50>,
               <mlp.dataset.MACLDataProvider at 0x31c5dd0>)),
             ('LLAx',
              (<mlp.layers.MLP at 0x31c5b10>,
               <mlp.dataset.MACLDataProvider at 0x677c210>)),
             ('RHAx',
              (<mlp.layers.MLP at 0x677c290>,
               <mlp.dataset.MACLDataProvider at 0x677c0d0>)),
             ('LHAx',
              (<mlp.layers.MLP at 0x677c150>,
               <mlp.dataset.MACLDataProvider at 0x67696d0>)),
             ("['RLAx', 'LLAx']",
              (<mlp.layers.MLP at 0x6769750>,
               <mlp.dataset.MACLDataProvider at 0x67698d0>)),
             ("['RLAx', 'RHAx']",
              (<mlp.layers.MLP at 0x6769950>,
               <mlp.dataset.MACLDataProvider at 0x6769ad0>)),
             ("['RLAx', 'LHAx']",
              (<mlp.layers.MLP at 0x6769b50>,
               <mlp.dataset.MACLDataProvider at 0x6769cd0>)),
             ("['LLAx', 'RHAx']",
              (<mlp.

In [31]:
print len(g)
learning_rate =0.01
max_epochs = 1000
lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)


15


In [14]:
# player one
phi1 = list()
# 1,2,3,4 
# v({1}) - v(not)
g['RLAx'][1].reset()
phi1 += [optimiser.validate(g['RLAx'][0], g['RLAx'][1] )[1]]*6

In [32]:
g["['RLAx', 'LLAx']"][1].reset()
print g["['RLAx', 'LLAx']"][1].x.shape
optimiser.validate(g["['RLAx', 'LLAx']"][0], g["['RLAx', 'LLAx']"][1] )

(1140, 250)


(1.0720617841854558, 0.68421052631578949)

In [39]:
def sor(corr, n):
    return [i for i in corr if i in n]

In [43]:
from itertools import permutations as permutation


# optimiser.validate is equivalent to the value func v()
# in wikipedia

d = ['RLAx', 'LLAx', 'RHAx', 'LHAx']
dp = list(permutation(d))
p1 = 'RLAx'
valsp1 = list()
for i, elem in enumerate(dp):
    ind =  dp[i].index(p1)
    if len(list(dp[i][:ind + 1]) ) == 1:
        predlistp = str( (dp[i][ind])  )
        print predlistp
    else:
        predlistp = str( sor(d,list(dp[i][:ind + 1]))  )
    if len(list(dp[i][:ind])) == 1:
        predlist = list(dp[i][:ind])[0]
    else:
        predlist =  sor(d,list(dp[i][:ind]))
    if predlist == []:
        mod , val = g[predlistp]
        val.reset()
        valsp1.append(optimiser.validate(mod,val)[1])
    else:
        mod , val = g[predlistp]
        mod2 , val2 = g[str(predlist)]
        val.reset()
        val2.reset()
        valsp1.append(optimiser.validate(mod,val)[1]
                      - optimiser.validate(mod2,val2)[1])


RLAx
RLAx
RLAx
RLAx
RLAx
RLAx


In [46]:
sum(valsp1) / len(valsp1)

0.22733918128654965

In [47]:
d = ['RLAx', 'LLAx', 'RHAx', 'LHAx']
dp = list(permutation(d))
p1 = 'LLAx'
valsp1 = list()
for i, elem in enumerate(dp):
    ind =  dp[i].index(p1)
    if len(list(dp[i][:ind + 1]) ) == 1:
        predlistp = str( (dp[i][ind])  )
        print predlistp
    else:
        predlistp = str( sor(d,list(dp[i][:ind + 1]))  )
    if len(list(dp[i][:ind])) == 1:
        predlist = list(dp[i][:ind])[0]
    else:
        predlist =  sor(d,list(dp[i][:ind]))
    if predlist == []:
        mod , val = g[predlistp]
        val.reset()
        valsp1.append(optimiser.validate(mod,val)[1])
    else:
        mod , val = g[predlistp]
        mod2 , val2 = g[str(predlist)]
        val.reset()
        val2.reset()
        valsp1.append(optimiser.validate(mod,val)[1]
                      - optimiser.validate(mod2,val2)[1])

LLAx
LLAx
LLAx
LLAx
LLAx
LLAx


In [48]:
sum(valsp1) / len(valsp1)

0.19722222222222219

In [50]:
d = ['RLAx', 'LLAx', 'RHAx', 'LHAx']
dp = list(permutation(d))
p1 = 'RHAx'
valsp1 = list()
for i, elem in enumerate(dp):
    ind =  dp[i].index(p1)
    if len(list(dp[i][:ind + 1]) ) == 1:
        predlistp = str( (dp[i][ind])  )
        print predlistp
    else:
        predlistp = str( sor(d,list(dp[i][:ind + 1]))  )
    if len(list(dp[i][:ind])) == 1:
        predlist = list(dp[i][:ind])[0]
    else:
        predlist =  sor(d,list(dp[i][:ind]))
    if predlist == []:
        mod , val = g[predlistp]
        val.reset()
        valsp1.append(optimiser.validate(mod,val)[1])
    else:
        mod , val = g[predlistp]
        mod2 , val2 = g[str(predlist)]
        val.reset()
        val2.reset()
        valsp1.append(optimiser.validate(mod,val)[1]
                      - optimiser.validate(mod2,val2)[1])

sum(valsp1) / len(valsp1)

RHAx
RHAx
RHAx
RHAx
RHAx
RHAx


0.14122807017543862

In [51]:
d = ['RLAx', 'LLAx', 'RHAx', 'LHAx']
dp = list(permutation(d))
p1 = 'LHAx'
valsp1 = list()
for i, elem in enumerate(dp):
    ind =  dp[i].index(p1)
    if len(list(dp[i][:ind + 1]) ) == 1:
        predlistp = str( (dp[i][ind])  )
        print predlistp
    else:
        predlistp = str( sor(d,list(dp[i][:ind + 1]))  )
    if len(list(dp[i][:ind])) == 1:
        predlist = list(dp[i][:ind])[0]
    else:
        predlist =  sor(d,list(dp[i][:ind]))
    if predlist == []:
        mod , val = g[predlistp]
        val.reset()
        valsp1.append(optimiser.validate(mod,val)[1])
    else:
        mod , val = g[predlistp]
        mod2 , val2 = g[str(predlist)]
        val.reset()
        val2.reset()
        valsp1.append(optimiser.validate(mod,val)[1]
                      - optimiser.validate(mod2,val2)[1])

sum(valsp1) / len(valsp1)

LHAx
LHAx
LHAx
LHAx
LHAx
LHAx


0.12280701754385966